In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical

In [2]:
import os
os.getcwd()

'/home/jango/Documents/EmergenTech'

In [42]:
Exprs = pd.read_csv("L1000dataset.csv")
Classes = pd.read_csv("Labels.csv")

In [24]:
Exprs.head()

,Unnamed: 0,A549_Ubenimex_33387,A549_Lamivudine_34563,A549_Tenofovir_32462,A549_Tenofovir_32659,A549_Indinavir_31762,A549_Zidovudine_34306,A549_Zalcitabine_34734,A549_Stavudine_31777,A549_Stavudine_34737,...,PC3_252916-29-3_46852,PC3_Cerivastatin_46118,PC3_Entacapone_47341,PC3_Cilastatin_46124,PC3_Triacsin_45231,PC3_Radicicol_43903,PC3_Radicicol_43935,PC3_Radicicol_67973,PC3_Radicicol_67994,PC3_Atorvastatin_45069
0,PSME1,-0.71402,-0.261000,-0.233967,-0.20424,0.38660,-0.105767,0.35075,0.38545,-0.19575,...,0.62534,0.229225,0.04358,0.254875,-0.048750,0.499217,0.965312,-1.995680,-0.708744,-0.282367
1,ATF1,-0.83916,-0.865733,-0.229533,-1.34262,-3.33905,0.147700,-1.40100,-2.80180,0.39535,...,0.30902,-0.310350,1.64042,0.771850,0.659525,0.213408,-0.612675,-0.219567,-1.703590,-0.492350
2,RHEB,-0.26298,-0.962867,-0.514633,-0.64016,0.43700,0.263433,1.14250,0.41765,-0.26020,...,0.61842,0.304250,0.17078,0.113750,0.313550,0.608542,-0.698775,0.468811,1.272740,0.053400
3,FOXO3,-0.23738,-0.539067,0.389833,-0.06742,0.05010,0.454067,0.70280,-0.41480,0.89450,...,0.95940,3.096830,0.61976,-0.364675,-0.261175,0.079492,0.709450,-0.376867,1.460910,1.628220
4,RHOA,-0.06352,0.049533,0.288767,1.40810,0.17155,-0.012300,0.76760,0.69215,0.87440,...,0.60308,4.557550,-0.90880,0.236425,0.179150,-1.182780,0.391937,-1.543410,-0.176822,5.338580


In [21]:
Genes = Exprs.ix[:,0]

In [36]:
data = Exprs.ix[:,1:].values

In [48]:
Y = Classes.ix[:,1].values

In [49]:
Y

array([0, 0, 0, ..., 5, 5, 5])

In [50]:
len(Y)

2346

In [51]:
records = len(Y)

shuffle = np.arange(records)
np.random.shuffle(shuffle)
test_fraction = 0.9

In [52]:
shuffle

array([2140, 1322,  383, ..., 1888,  716, 1489])

In [54]:
train_split, test_split = shuffle[:int(records*test_fraction)], shuffle[int(records*test_fraction):]

In [55]:
train_split

array([2140, 1322,  383, ...,  400, 1394, 1469])

In [58]:
data.shape

(978, 2346)

In [59]:
trainX, trainY = data[:,train_split], to_categorical(Y[train_split], 6)
testX, testY = data[:,test_split], to_categorical(Y[test_split], 6)

trainX = trainX.T
testX = testX.T

In [69]:
trainX.T.shape

(2111, 978)

In [66]:
trainX[0].shape

(2111,)

In [65]:
trainY[0].shape

(6,)

In [71]:
trainX[0].shape

(978,)

# Building MedNet

In [226]:
# Network building
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    # Inputs
    net = tflearn.input_data([None, 978])

    # Hidden layer(s)
    h1 = tflearn.fully_connected(net, 200, activation='ReLU')
    # net = tflearn.batch_normalization(h1)
    net = tflearn.dropout(h1, 0.2)
    net = tflearn.fully_connected(net, 45, activation='ReLU')
    net = tflearn.dropout(net, 0.9)
    net = tflearn.batch_normalization(net)

    # Output layer
    net = tflearn.fully_connected(net, 6, activation='softmax')
    net = tflearn.regression(net, optimizer='sgd', 
                             learning_rate=0.089, 
                             loss='categorical_crossentropy')
    
    model = tflearn.DNN(net)
    w = model.get_weights(h1.W)
    
    return (model, w)

### Initializing the model

In [227]:
model, w = build_model()

### Training

In [228]:
# Training
model.fit(trainX, trainY, validation_set=0.1, show_metric=True, batch_size=64, n_epoch=8)

Training Step: 239  | total loss: 1.01791 | time: 0.156s
| SGD | epoch: 008 | loss: 1.01791 - acc: 0.6497 -- iter: 1856/1899
Training Step: 240  | total loss: 1.02277 | time: 1.164s
| SGD | epoch: 008 | loss: 1.02277 - acc: 0.6504 | val_loss: 1.08452 - val_acc: 0.6038 -- iter: 1899/1899
--


### Get weights

In [229]:
w

array([[-0.00157063,  0.00448441, -0.00326898, ..., -0.00430286,
         0.0065139 ,  0.0238764 ],
       [ 0.02571013, -0.0173589 , -0.02983337, ...,  0.00394139,
        -0.03100454,  0.00435774],
       [-0.00148692,  0.02766705, -0.0010295 , ..., -0.01611471,
        -0.01484044,  0.00320762],
       ..., 
       [-0.00292515,  0.00814916, -0.01456198, ...,  0.0204108 ,
        -0.01442776, -0.02361646],
       [ 0.02458762,  0.03191415, -0.01981831, ..., -0.02489581,
         0.00790949,  0.00753898],
       [ 0.02865823,  0.0358302 ,  0.00555496, ...,  0.01518264,
        -0.02405457,  0.00930682]], dtype=float32)

In [230]:
weights_to_embedd = pd.DataFrame(w)

In [231]:
weights_to_embedd.to_csv("/home/jango/Documents/EmergenTech/Hidden_Layer_1_Weight.csv", index=False)